In [1]:
import matplotlib.pyplot as plt
import numpy as nm
import pandas as pd

In [2]:
folder = 'C:\\Users\\vswen\\Documents\\1. Biomedische Technologie\\BMT JAAR 5\\Kwart 4\\4. Data\\CTRPv2.0_2015_ctd2_ExpandedDataset\\'


In [3]:
#importing datasets
complete_df = pd.read_csv(f"{folder}v20.data.fingerprints.txt", sep="\t")
complete_df.fillna(complete_df.mean(), inplace=True)

#grenswaarden ec50 aangeven
complete_df['ec50_molair'] = -nm.log10(complete_df['ec50_molair'])
condition = (complete_df['ec50_molair'] < 1) | (complete_df['ec50_molair'] > 10)
complete_df=complete_df[~condition]

C:\Users\vswen\AppData\Local\Temp\ipykernel_15192\3024066475.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  complete_df.fillna(complete_df.mean(), inplace=True)
C:\Users\vswen\AppData\Roaming\anaconda3\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
#ECFP uitlezen
ECFP_string = complete_df['ECFP']
ECFP_list = []
import ast
for string in complete_df['ECFP']:
    ECFP_single_list = ast.literal_eval(string)
    ECFP_list.append(ECFP_single_list)

In [5]:
#extracting independent and dependent variable
x=ECFP_list
y = complete_df['ec50_molair'].values
print('y=', y)

y= [9.87824964 9.75860969 9.91051887 9.17434816 8.55165478 8.14747271
 7.97837982 8.22021806 7.32257936 7.55637684 7.52963757 7.39410587
 7.6705313  7.86115472 8.06405074 7.8379193  7.90763278 8.00337079
 7.02485145 7.0805511  7.474078   6.600372   7.14399673 7.14012004
 7.23431301 6.81107136 7.00286775 7.11952094 6.71969129 7.20416425
 6.8726443  6.82001238 6.57655043 6.70201513 6.49916268 6.6667219
 6.99487117 6.49235756 7.35864876 6.41357527 6.53503479 6.55478262
 6.80647327 6.46322491 6.17746792 6.2821067  6.43209888 6.29818686
 6.25316344 6.12197721 6.36040714 6.16404616 6.19343921 6.09904609
 6.30261421 5.85993903 6.16761988 6.06617708 6.41198592 6.13666682
 5.80930103 5.94717067 6.35001835 5.84272534 5.97726562 5.92818719
 6.10140591 5.69925556 6.45297299 5.75300687 5.68731536 5.29571247
 5.986187   6.15520554 5.92664443 5.71387782 5.91407537 5.71087027
 5.58757763 6.57711339 5.33616141 5.65632768 5.15262715 5.57552988
 5.93480469 5.71056475 5.54643757 5.43370526 5.52229246 5.85

In [6]:
#splitting dataset into training and test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)

In [13]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in nm.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in nm.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [14]:
# Fitting Decision Tree classifier to the training set | friedman_mse
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
#regressor = RandomForestRegressor(n_estimators=1024, random_state=42) #nestimators is requorednumber of trees in the trandom forest
regressor = RandomizedSearchCV(estimator = RandomForestRegressor(random_state = 42), param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
regressor.fit(x_train,y_train)

OSError: [Errno 22] Invalid argument

In [10]:
# Predicting the test result
y_pred = regressor.predict(x_test)

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [11]:
# Predicting the test result
y_pred = regressor.predict(x_test)

# Errors berekenen
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae=mean_absolute_error(y_test, y_pred)
print('mean absolute error is ', mae)

mse = mean_squared_error(y_test, y_pred)
import math
rmse = math.sqrt(mse)
print('mean squared error is ', mse)
print('root mean squared error is ', rmse)

slope, intercept = nm.polyfit(y_test,y_pred,1)
line = slope * nm.array(y_test)+ intercept
r2 = r2_score(y_test, y_pred)

print('y_test =',y_test)
print('y_pred =',y_pred)

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [12]:
plt.scatter(y_test,y_pred)
plt.plot(y_test, line, color='red', label='line of current best fit')
plt.xlabel('y_test')
plt.ylabel('y_pred')
plt.title('Scatterplot with Line of Best Fit (R2 = {:.2f})'.format(r2))
plt.show()

NameError: name 'y_pred' is not defined